## Linear correlation of non-explicitely encoded variables with observations and LSTM output (panel D)

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from definitions import ROOT_DIR
import sklearn.linear_model
from envs.environment_factory import EnvironmentFactory
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import VecNormalize
from functions_notebook import make_parallel_envs
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import scipy.stats as stats


# Load the dataset and run linear regressions

In [ ]:
df1 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_ccw", "data.hdf"))
df2 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_cw", "data.hdf"))

df = pd.concat((df1, df2)).reset_index()
df.keys()

In [ ]:
results_list = []
regression = sklearn.linear_model.LinearRegression()
for target in ["mass_1", "mass_2", "size_1", "size_2", "friction_0", "friction_1", "friction_2", "x_radius", "y_radius"]:
    for key in ["observation", "lstm_state_0", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]:
        X = np.array(df[key].to_list())
        y = df[target].to_numpy()
        cv = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
        cv_score = sklearn.model_selection.cross_val_score(regression, X, y, cv=cv)
        print("Key:", key, " target:", target,  "score:", cv_score)
        results_list.append({"input": key, "target": target, "score": cv_score})


In [ ]:
classification = sklearn.linear_model.LogisticRegression(max_iter=10_000)
target = "task"
for key in ["observation", "lstm_state_0", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]:
    X = np.array(df[key].to_list())
    y = df[target].to_numpy()
    cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = sklearn.model_selection.cross_val_score(classification, X, y, cv=cv)
    print("Key:", key, " target:", target,  "score:", cv_score)
    results_list.append({"input": key, "target": target, "score": cv_score})

In [ ]:
regression = sklearn.linear_model.LinearRegression()
for target in ["hand_pos", "hand_vel"]:
    for key in ["observation", "lstm_state_0", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]:
        X = np.array(df[key].to_list())
        y = np.array(df[target].to_list())

        cv = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
        cv_score = sklearn.model_selection.cross_val_score(regression, X, y, cv=cv)

        print("Key:", key, " target:", target,  "score:", cv_score)
        results_list.append({"input": key, "target": target, "score": cv_score})

In [ ]:
results_list_mean_sem = [{
    "input": el["input"],
    "target": el["target"],
    "score_mean": np.mean(el["score"]),
    "score_std": np.std(el["score"])
    }
    for el in results_list
]
results_df = pd.DataFrame(results_list_mean_sem)
results_df

In [ ]:
layers_list = ["observation", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]
layers_name_list = ["Observation", "LSTM state", "LSTM out", "Layer 1 out", "Layer 2 out", "Action"]
targets_list = ["mass_1", "mass_2", "size_1", "size_2", "friction_0", "friction_1", "friction_2", "x_radius", "y_radius", "task", "hand_pos", "hand_vel"]
targets_name_list = ["Mass 1", "Mass 2", "Size 1", "Size 2", "Friction 0", "Friction 1", "Friction 2", "Radius x", "Radius y", "Task", "Joint pos", "Joint vel"]

# Create a colormap with distinct colors
num_value_types = len(set([value.split("_")[0] for value in targets_list]))
cmap = plt.get_cmap('brg')
colors = [cmap(i % num_value_types) for i in range(len(targets_list))]

fig, ax = plt.subplots(1)
for target, target_name, c in zip(targets_list, targets_name_list, colors):
    score_list = []
    for layer in layers_list:
        score = results_df[(results_df.input == layer) & (results_df.target == target)].score_mean.item()
        score_list.append(score)
    score_vec = np.array(score_list) / max(score_list)
    ax.plot(score_vec, label=target_name, color=c)
ax.legend()
ax.set_xticks(range(6), labels=layers_name_list, rotation=30)
ax.set_ylabel("Rescaled score")
ax.legend(bbox_to_anchor=(1, 1))
plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_4", "layer_encoding.png"), format="png", dpi=600, bbox_inches="tight")
plt.show()
    

In [ ]:
layers_list = ["observation", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]
layers_name_list = ["Observation", "LSTM state", "LSTM out", "Layer 1 out", "Layer 2 out", "Action"]
targets_list = ["mass_1", "mass_2", "size_1", "size_2", "friction_0", "friction_1", "friction_2", "x_radius", "y_radius", "task", "hand_pos", "hand_vel"]
targets_name_list = ["Mass 1", "Mass 2", "Size 1", "Size 2", "Friction 0", "Friction 1", "Friction 2", "Radius x", "Radius y", "Task", "Joint pos", "Joint vel"]

# Create a colormap with distinct colors
num_value_types = len(set([value.split("_")[0] for value in targets_list]))
cmap = plt.get_cmap('tab20')
colors = [cmap(i % num_value_types) for i in range(len(targets_list))]

fig, ax = plt.subplots(1)
for target, target_name, c in zip(targets_list, targets_name_list, colors):
    score_list = []
    for layer in layers_list:
        score = results_df[(results_df.input == layer) & (results_df.target == target)].score_mean.item()
        score_list.append(score)
    score_vec = np.array(score_list) / max(score_list)
    ax.plot(score_vec, label=target_name, color=c)
ax.legend()
ax.set_xticks(range(6), labels=layers_name_list, rotation=30)
ax.set_ylabel("Rescaled score")
ax.legend(bbox_to_anchor=(1, 1))
plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_4", "layer_encoding.png"), format="png", dpi=600, bbox_inches="tight")
plt.show()

In [ ]:
layers_list = ["observation", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]
layers_name_list = ["Observation", "Memory", "LSTM output", "Layer 1", "Layer 2", "Action"]
targets_list = ["mass_1", "mass_2", "size_1", "size_2", "friction_0", "friction_1", "friction_2", "x_radius", "y_radius", "task", "hand_pos", "hand_vel"]
targets_name_list = ["Mass 1", "Mass 2", "Size 1", "Size 2", "Friction 0", "Friction 1", "Friction 2", "Radius x", "Radius y", "Task", "Joint pos", "Joint vel"]

# Create a pivot table to reshape the data
pivot_data = results_df.pivot(index="target", columns="input", values="score_mean").loc[targets_list]
pivot_std = results_df.pivot(index="target", columns="input", values="score_std").loc[targets_list]

# Set the width of each bar
bar_width = 0.15

# Create an array of x values for the bars
x = np.arange(len(targets_list))

# Create a grouped barplot
fig, ax = plt.subplots(figsize=(5, 3.5))
cmap = get_cmap("coolwarm")


for i, layer in enumerate(layers_list):
    ax.bar(x + i * bar_width, pivot_data[layer], bar_width, yerr=pivot_std[layer], label=layers_name_list[i], color=cmap((i) / (len(layers_list))), alpha=0.9)

# Set x-axis labels and tick positions
ax.set_xticks(x + (len(pivot_data.columns) / 2) * bar_width)
ax.set_xticklabels(targets_name_list, rotation=45, ha='right')

# Set labels and title
# ax.set_xlabel('Encoded quantity')
ax.set_ylabel('Encoding score', fontsize=12)
# ax.set_title('Encoding Value by Layer for Each Quantity')

# Add a legend
ax.legend()

# Show the plot
plt.tight_layout()
plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_4", "layer_encoding_barplot.png"), format="png", dpi=600, bbox_inches="tight")

plt.show()